In [3]:
import pandas as pd
import geopandas as gpd
import numpy as np
from shapely.geometry import Polygon
from dbfread import DBF
from jenkspy import jenks_breaks
import matplotlib.pyplot as plt

In [4]:
# Verificar columnas del DF, para ver columnas a sumar, y columnas a recalcular

# Asegúrate de que pandas muestre todas las columnas y expanda el ancho de la visualización
pd.set_option('display.max_columns', None)  # Muestra todas las columnas
#pd.set_option('display.max_colwidth', None)  # No limite el ancho de las columnas
pd.set_option('display.expand_frame_repr', False)  # Evita el corte de DataFrames grandes

In [45]:
aforos_ruta = r"Z:\AMIM\DSS\Lázaro Cárdenas\250210 Lázaro Cardenas.xlsx"

lateral_OP = pd.read_excel(aforos_ruta, 'lateral_OP')
lateral_PO = pd.read_excel(aforos_ruta, 'lateral_PO')
central_OP = pd.read_excel(aforos_ruta, 'central_OP')
central_PO = pd.read_excel(aforos_ruta, 'central_PO') # Falló y contó mal

In [37]:
lateral_OP

,FECHA,HORA,VELOCIDAD ENTRADA,VELOCIDAD SALIDA,ACERCAMIENTO
0,2025-02-09,03:06:00,59,34.0,+
1,2025-02-09,03:06:00,58,31.0,+
2,2025-02-10,06:11:00,50,33.0,+
3,2025-02-10,06:11:00,58,36.0,+
4,2025-02-10,06:11:00,36,37.0,+
...,...,...,...,...,...
2615,2025-02-10,08:03:00,66,NaN,-
2616,2025-02-10,08:03:00,37,NaN,-
2617,2025-02-10,08:03:00,42,NaN,-
2618,2025-02-10,08:03:00,34,NaN,-


In [47]:
def procesar_aforos(df, nombre_columna):
    df['HORA'] = df['HORA'].astype(str).str.strip()  # Limpiar espacios y caracteres extra
    df['HORA'] = pd.to_datetime(df['HORA'], format='%H:%M:%S', errors='coerce')  # Convertir correctamente considerando segundos
    df = df.dropna(subset=['HORA'])  # Eliminar filas con errores de conversión
    df = df[(df['HORA'].dt.hour >= 6) & (df['HORA'].dt.hour < 9)]
    df['Intervalo'] = df['HORA'].dt.floor('15min')
    return df.groupby('Intervalo').size().rename(nombre_columna)
    

# Procesar cada DataFrame
lop = procesar_aforos(lateral_OP, 'LOP')
cop = procesar_aforos(central_OP, 'COP')
lpo = procesar_aforos(lateral_PO, 'LPO')
cpo = procesar_aforos(central_PO, 'CPO')

# Unir los resultados en un solo DataFrame
resultado = pd.concat([lop, cop, cpo, lpo], axis=1).fillna(0)
resultado = resultado.astype(int)  # Asegurar que los valores sean enteros

resultado


,LOP,COP,CPO,LPO
Intervalo,,,,
1900-01-01 06:00:00,53,114,413,94
1900-01-01 06:15:00,201,351,443,130
1900-01-01 06:30:00,252,258,485,130
1900-01-01 06:45:00,378,378,508,141
1900-01-01 07:00:00,371,292,547,132
1900-01-01 07:15:00,398,413,67,323
1900-01-01 07:30:00,450,392,172,365
1900-01-01 07:45:00,432,464,541,274
1900-01-01 08:00:00,83,27,541,254


In [49]:
resultado['Intervalo'] = resultado.index

In [51]:
aforos_ruta_salida = r"Z:\AMIM\DSS\Lázaro Cárdenas\250210_Lázaro_Cardenas_resultados.xlsx"

resultado.to_excel(aforos_ruta_salida, index = False)

In [43]:
# Convertir Intervalo a formato de hora para facilitar análisis
resultado["Intervalo"] = pd.to_datetime(resultado["Intervalo"], format="%H:%M:%S").dt.time

# Calcular la relación entre COP y LOP
resultado["Factor_Central"] = resultado["COP"] / resultado["LOP"].replace(0, 1)  # Evitar división por cero

# Aplicar este factor a LPO para estimar CPO corregido
resultado["CPO_Corregido"] = resultado["LPO"] * resultado["Factor_Central"]

# Ver resultado
print(resultado[["Intervalo", "CPO", "CPO_Corregido"]])

                    Intervalo  CPO  CPO_Corregido
Intervalo                                        
1900-01-01 06:00:00  06:00:00  413     202.188679
1900-01-01 06:15:00  06:15:00  443     227.014925
1900-01-01 06:30:00  06:30:00  485     133.095238
1900-01-01 06:45:00  06:45:00  508     141.000000
1900-01-01 07:00:00  07:00:00  547     103.892183
1900-01-01 07:15:00  07:15:00   67     335.173367
1900-01-01 07:30:00  07:30:00  172     317.955556
1900-01-01 07:45:00  07:45:00  541     294.296296
1900-01-01 08:00:00  08:00:00  541      82.626506
1900-01-01 08:15:00  08:15:00  114       0.000000


In [25]:

# Calcular la relación entre COP y LOP
resultado["Factor_Central"] = 937 / 684

# Aplicar este factor a LPO para estimar CPO corregido
resultado["CPO_Corregido_Manual"] = resultado["LPO"] * resultado["Factor_Central"]

# Ver resultado
print(resultado[["Intervalo", "CPO","CPO_Corregido", "CPO_Corregido_Manual"]])

                    Intervalo  CPO  CPO_Corregido  CPO_Corregido_Manual
Intervalo                                                              
1900-01-01 07:00:00  07:00:00  547     103.892183            180.824561
1900-01-01 07:15:00  07:15:00   67     335.173367            442.472222
1900-01-01 07:30:00  07:30:00  172     317.955556            500.007310
1900-01-01 07:45:00  07:45:00  541     294.296296            375.347953
1900-01-01 08:00:00  08:00:00  541      82.626506            347.950292
1900-01-01 08:15:00  08:15:00  114       0.000000             71.233918


In [134]:
import pandas as pd
import numpy as np

# Suponiendo que ya tienes el dataframe llamado 'resultado'
# Reemplazamos los valores en LOP y COP a partir de las 7:07 y 7:18 respectivamente

# Ajustar el conteo para LOP, comenzando a las 7:18
lop_start_time = '1900-01-01 07:18:00'
cop_start_time = '1900-01-01 07:07:00'

resultado['LOP'] = np.where(resultado.index >= lop_start_time, resultado['LOP'], np.nan)  # Ajustar LOP
resultado['COP'] = np.where(resultado.index >= cop_start_time, resultado['COP'], np.nan)  # Ajustar COP

# Los valores manuales de 7:50 a 8:05
manual_lop = 678
manual_cop = 937

# Ubicación de los índices correspondientes a las 7:50 y 8:05
start_time = '1900-01-01 07:50:00'
end_time = '1900-01-01 08:05:00'

# Calcular los conteos por intervalo
lop_intervals = resultado.loc[start_time:end_time, 'LOP']
cop_intervals = resultado.loc[start_time:end_time, 'COP']

# Ajuste de la suma total en esos intervalos
lop_sum = lop_intervals.sum()
cop_sum = cop_intervals.sum()

# Ajustar la distribución proporcionalmente
lop_factor = manual_lop / lop_sum
cop_factor = manual_cop / cop_sum

# Aplicar la corrección multiplicando por el factor
resultado.loc[start_time:end_time, 'LOP'] *= lop_factor
resultado.loc[start_time:end_time, 'COP'] *= cop_factor

# Interpolar los valores en la columna, evitando 'inplace=True'
resultado['LOP'] = resultado['LOP'].interpolate(method='linear')
resultado['COP'] = resultado['COP'].interpolate(method='linear')

# Rellenar los valores de NaN antes de las 7:50 usando 'ffill' o 'bfill' si es necesario
resultado['LOP'] = resultado['LOP'].fillna(method='bfill')
resultado['COP'] = resultado['COP'].fillna(method='bfill')

# Ver el dataframe corregido
print(resultado)


                       LOP    COP  CPO  LPO
Intervalo                                  
1900-01-01 07:00:00  472.0  500.0   21  163
1900-01-01 07:15:00  472.0  500.0   47  157
1900-01-01 07:30:00  472.0  503.0   51  167
1900-01-01 07:45:00  490.0  416.0   41  275
1900-01-01 08:00:00  678.0  937.0   55  232
1900-01-01 08:15:00  493.0  525.0   35  226
1900-01-01 08:30:00  463.0  525.0   48  204
1900-01-01 08:45:00  486.0  498.0   47  216


C:\Users\AMIM GTAC\AppData\Local\Temp\ipykernel_13692\3836529585.py:43: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  resultado['LOP'] = resultado['LOP'].fillna(method='bfill')
C:\Users\AMIM GTAC\AppData\Local\Temp\ipykernel_13692\3836529585.py:44: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  resultado['COP'] = resultado['COP'].fillna(method='bfill')
